<a href="https://colab.research.google.com/github/sushirito/Methylmercury/blob/main/Diffusion_Boltzmann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This code was inspired by: https://github.com/lbm-principles-practice/code/blob/master/chapter9/shanchen.cpp



In [1]:
import numpy as np
import plotly.graph_objects as go
import os
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import seaborn as sns

# Lattice parameters
nx, ny, nz = 20, 20, 20  # Domain size matching Lx, Ly, Lz
nsteps = 1000  # Number of time steps matching the given t = 4000
noutput = 100 # Output frequency

# D3Q7 model parameters
npop = 7  # Number of populations
cx = np.array([0, 1, -1, 0, 0, 0, 0])
cy = np.array([0, 0, 0, 1, -1, 0, 0])
cz = np.array([0, 0, 0, 0, 0, 1, -1])
weight = np.array([1 / 4, 1 / 8, 1 / 8, 1 / 8, 1 / 8, 1 / 8, 1 / 8])  # Weights for D3Q7

# Diffusion parameters
D0 = 0.5  # Molecular diffusion coefficient [l.u.]^2 / [t.u.]
Lambda_eo = 1 / 32  # As given in the paper
cs_sq = 1 / 3  # Square of the lattice speed of sound

lambda_o = -2 * cs_sq / (2 * D0 + cs_sq)
Lambda_o = -(1 / 2 + 1 / lambda_o)
Lambda_e = Lambda_eo / Lambda_o
lambda_e = -2 / (2 * Lambda_e + 1)

# Initialize arrays
rho = np.zeros(nx * ny * nz)  # Density (concentration)
f1 = np.zeros((nx * ny * nz, npop))  # Distribution functions
f2 = np.zeros((nx * ny * nz, npop))  # Temporary array for streaming

def create_elementary_pattern(Le):
    pattern = np.zeros((Le, Le, Le), dtype=bool)
    radius = Le // 2
    corners = [(0, 0, 0), (0, 0, Le), (0, Le, 0), (Le, 0, 0),
               (0, Le, Le), (Le, 0, Le), (Le, Le, 0), (Le, Le, Le)]

    for corner in corners:
        cx, cy, cz = corner
        x, y, z = np.ogrid[0:Le, 0:Le, 0:Le]
        dist_from_corner = (x - cx) ** 2 + (y - cy) ** 2 + (z - cz) ** 2
        pattern |= (dist_from_corner <= radius ** 2)

    return pattern

def create_porous_medium(N, Le):
    elementary_pattern = create_elementary_pattern(Le)
    Lm = int(N * Le)
    medium = np.tile(elementary_pattern, (int(np.ceil(N)), int(np.ceil(N)), int(np.ceil(N))))
    return medium[:Lm, :Lm, :Lm]

def calculate_porosity(medium):
    return 1 - np.mean(medium)

# Parameters
Le = 10
N = 2  # Adjusted for larger domain size

# Create porous medium
solid = create_porous_medium(N, Le)

# Calculate porosity
porosity = calculate_porosity(solid)
print(f"Porosity: {porosity:.3f}")

# Equilibrium functions
def equilibrium(rho_local):
    feq = np.zeros(npop)
    feq[0] = weight[0] * rho_local
    for q in range(1, npop):
        feq[q] = weight[q] * rho_local
    return feq



def compute_moments():
    global rho
    rho[:] = np.sum(f1, axis=1)

def collision_and_streaming():
    global f1, f2
    for z in range(nz):
        for y in range(ny):
            for x in range(nx):
                k = z * nx * ny + y * nx + x
                if solid[z, y, x] == 0:  # Fluid node
                    feq = equilibrium(rho[k])
                    for i in range(npop):
                        f_plus = 0.5 * (f1[k, i] + f1[k, npop - 1 - i])
                        f_minus = 0.5 * (f1[k, i] - f1[k, npop - 1 - i])
                        n_plus = f_plus - feq[i]
                        n_minus = f_minus

                        x2 = (x + cx[i] + nx) % nx
                        y2 = (y + cy[i] + ny) % ny
                        z2 = (z + cz[i] + nz) % nz
                        k2 = z2 * nx * ny + y2 * nx + x2

                        f2[k2, i] = f1[k, i] + lambda_e * n_plus + lambda_o * n_minus

    f1, f2 = f2, f1

# Initialization
def initialize_diffusion():
    global f1, f2, rho

    # Set initial concentration at the plane x = x0
    x0 = nx//2
    C0 = 1.0
    rho[:] = 0  # Initialize with zero concentration

    # Set initial concentration at the plane x = x0
    for z in range(nz):
        for y in range(ny):
            k = z * nx * ny + y * nx + x0
            rho[k] = C0  # Set initial concentration at x = x0

    # Initialize f1 and f2
    f1.fill(0)
    f2.fill(0)
    for k in range(nx * ny * nz):
        if not solid.flatten()[k]:
            feq = equilibrium(rho[k])
            f1[k] = feq
            f2[k] = feq

def analytical_solution(x, t, D_p, C0, x0):
    return (C0 / np.sqrt(max(4*np.pi*D_p*t, 1e-10))) * np.exp(-np.maximum((x-x0)**2 / (4*D_p*t), 0))

def calc_mean_concentration(rho):
    rho_3d = rho.reshape((nz, ny, nx))
    phi_x = np.sum(rho_3d > 0, axis=(0, 1))  # Number of liquid sites per x-plane
    C = np.zeros(nx)
    non_zero_phi = phi_x > 0
    C[non_zero_phi] = np.sum(rho_3d[:, :, non_zero_phi], axis=(0, 1)) / (phi_x[non_zero_phi] * ny * nz)
    return C

def fit_diffusion_coefficient(x, C, t, x0):
    def fit_func(x, D_p, C0):
        return analytical_solution(x, t, D_p, C0, x0)
    try:
        # Provide initial guesses and bounds
        initial_guess = [D0, np.max(C)]  # Use D0 as initial guess for D_p
        bounds = ([D0/100, 0], [D0*100, np.inf])  # Constrain D_p and C0
        popt, _ = curve_fit(fit_func, x, C, p0=initial_guess, bounds=bounds, maxfev=10000)
        D_p, C0 = popt
    except RuntimeError as e:
        print(f"Error in curve_fit: {str(e)}. Using default values.")
        D_p, C0 = D0, np.max(C)  # Use initial values as fallback
    return D_p, C0


def calc_l2_error(C_lbm, C_analytical):
    return np.sqrt(np.mean((C_lbm - C_analytical)**2))

# Assuming output_dir is defined as the directory where you want to save the plots
output_dir = 'output_plots'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def save_html(fig, filename):
    filepath = os.path.join(output_dir, filename)
    fig.write_html(filepath)
    print(f"Saved: {filepath}")

def save_png(fig, filename):
    filepath = os.path.join(output_dir, filename)
    fig.savefig(filepath)
    plt.close(fig)
    print(f"Saved: {filepath}")

def concentration_plot_3d(rho, step):
    # Creating meshgrid for 3D plot
    x, y, z = np.meshgrid(np.arange(nx), np.arange(ny), np.arange(nz))
    scalar_field = rho.reshape((nz, ny, nx)).flatten()

    # Create a 3D scatter plot
    fig = go.Figure()

    # Add liquid nodes with normal colors
    fig.add_trace(go.Scatter3d(
        x=x.flatten()[solid.flatten() == 0],
        y=y.flatten()[solid.flatten() == 0],
        z=z.flatten()[solid.flatten() == 0],
        mode='markers',
        marker=dict(
            size=4,
            color=scalar_field[solid.flatten() == 0],
            colorscale='Plasma',
            opacity=0.8,
            colorbar=dict(title='Concentration'),
            cmin=np.min(scalar_field),
            cmax=np.max(scalar_field)
        ),
        name='Liquid Nodes'
    ))

    # Add solid nodes with gray color
    fig.add_trace(go.Scatter3d(
        x=x.flatten()[solid.flatten() == 1],
        y=y.flatten()[solid.flatten() == 1],
        z=z.flatten()[solid.flatten() == 1],
        mode='markers',
        marker=dict(
            size=4,
            color='gray',
            opacity=0
        ),
        name='Solid Nodes'
    ))

    # Update layout for better visualization
    fig.update_layout(
        title=f'Concentration Distribution at Step {step}',
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z')
        )
    )

    # Show the figure
    fig.show()

    # Save the plot as HTML
    save_html(fig, f'concentration_plot_3d_{step:05d}.html')

def plot_concentration_profile(x, C_lbm, C_analytical, step, output_dir):
    plt.figure(figsize=(10, 8))
    sns.set_style("whitegrid")
    sns.set_context("paper", font_scale=1.5)

    # Plot LBM data
    sns.lineplot(x=x, y=C_lbm, color='green', linewidth=2, label='LBM')

    # Plot analytical solution
    sns.lineplot(x=x, y=C_analytical, color='red', linewidth=2, linestyle='--', label='Ana.')

    plt.xlabel('x [lu]')
    plt.ylabel('C [mu].[lu]$^{-3}$')
    plt.title(f'Concentration Profile at Step {step}')
    plt.legend()

    # Set y-axis to scientific notation
    plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))

    # Adjust plot limits
    plt.xlim(0, nx)
    plt.ylim(0, max(C_lbm.max(), C_analytical.max()) * 1.1)

    # Add gridlines
    plt.grid(True, linestyle=':', alpha=0.7)

    # Save the plot
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'concentration_profile_{step:05d}.png'), dpi=300)
    plt.close()

def visualize_boundary_nodes(solid):
    fig = go.Figure(data=go.Volume(
        x=np.arange(nx).repeat(ny * nz),
        y=np.tile(np.arange(ny).repeat(nz), nx),
        z=np.tile(np.arange(nz), nx * ny),
        value=solid.flatten(),
        opacity=0.8,  # Change this value to make boundaries more/less visible
        isomin=0.5,
        isomax=1,
        surface_count=3,
        colorscale='Blues'
    ))

    fig.update_layout(
        title='Boundary Nodes Visualization',
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z')
        )
    )

    fig.show()
    save_html(fig, 'boundary_nodes_visualization.html')

def main_loop():
    initial_mass = np.sum(rho)
    mass_changes = []
    steps = []
    x0 = nx // 2

    for step in range(nsteps):
        compute_moments()
        collision_and_streaming()

        if step % noutput == 0:
            current_mass = np.sum(rho)
            mass_change = (current_mass - initial_mass) / initial_mass * 100
            mass_changes.append(mass_change)
            steps.append(step)

            C = calc_mean_concentration(rho)
            x = np.arange(nx)
            D_p, C0 = fit_diffusion_coefficient(x, C, step+1, x0)
            D_e = porosity * D_p

            C_analytical = analytical_solution(x, step+1, D_p, C0, x0)
            l2_error = calc_l2_error(C, C_analytical)

            print(f"Step {step}: Average concentration = {np.mean(rho[solid.flatten() == 0])}")
            print(f"Mass change: {mass_change:.6%}")
            print(f"Min concentration: {np.min(rho)}")
            print(f"Max concentration: {np.max(rho)}")
            print(f"Total mass: {current_mass}")
            print(f"Effective diffusion coefficient: {D_e:.6f}")
            print(f"L2 error: {l2_error:.6f}")

            concentration_plot_3d(rho, step)
            plot_concentration_profile(x, C, C_analytical, step, output_dir)

    final_mass = np.sum(rho)
    total_mass_change = (final_mass - initial_mass) / initial_mass
    print(f"Total mass change: {total_mass_change:.6%}")

# Visualize the boundary nodes once before running the simulation
visualize_boundary_nodes(solid)

# Initialize and run simulation
initialize_diffusion()
main_loop()

print("Simulation complete.")

Porosity: 0.488


Saved: output_plots/boundary_nodes_visualization.html
Step 0: Average concentration = 0.02151639344262295
Mass change: -7900.000000%
Min concentration: 0.0
Max concentration: 1.0
Total mass: 84.0
Effective diffusion coefficient: 0.011083
L2 error: 0.000000


Saved: output_plots/concentration_plot_3d_00000.html
Step 100: Average concentration = 0.04856154419901655
Mass change: -4810.781285%
Min concentration: -0.06081109961685762
Max concentration: 0.910790930365322
Total mass: 207.5687485971576
Effective diffusion coefficient: 0.002440
L2 error: 0.000082


Saved: output_plots/concentration_plot_3d_00100.html
Step 200: Average concentration = 0.048511623436969076
Mass change: -4814.936387%
Min concentration: -0.060278358172115146
Max concentration: 0.9109384673556081
Total mass: 207.4025445372671
Effective diffusion coefficient: 0.002440
L2 error: 0.000098


Saved: output_plots/concentration_plot_3d_00200.html
Step 300: Average concentration = 0.048529721609257316
Mass change: -4813.675302%
Min concentration: -0.06065524417885452
Max concentration: 0.9107846273239967
Total mass: 207.45298792260746
Effective diffusion coefficient: 0.002440
L2 error: 0.000111


Saved: output_plots/concentration_plot_3d_00300.html
Step 400: Average concentration = 0.04852959601800082
Mass change: -4813.431751%
Min concentration: -0.06045648801981354
Max concentration: 0.9108405978575476
Total mass: 207.4627299700074
Effective diffusion coefficient: 0.002440
L2 error: 0.000121


Saved: output_plots/concentration_plot_3d_00400.html
Step 500: Average concentration = 0.04852617550034877
Mass change: -4813.874174%
Min concentration: -0.06053913610022657
Max concentration: 0.910828822204975
Total mass: 207.4450330269939
Effective diffusion coefficient: 0.229963
L2 error: 0.000218


Saved: output_plots/concentration_plot_3d_00500.html


KeyboardInterrupt: 